# Methun Research — CNN-Transformer IDS with XAI

This notebook runs the **full pipeline** on Google Colab or a GCE VM:

| Step | What happens |
|------|-------------|
| **1. Setup** | Clone repo, install NVIDIA drivers + CUDA + deps, verify GPU |
| **2. Data** | Upload / mount CICIDS2017 dataset |
| **3. CNN-Transformer Training** | Train the hybrid model (+ IG & Grad-CAM) |
| **4. Enhanced Transformer Training** | Train the grouped-token model |
| **5. SHAP** | GradientExplainer feature importance + plots |
| **6. Test Evaluation** | Held-out test set metrics (20 % unseen data) |
| **7. Results** | Display all XAI outputs side by side |

> **Colab:** Select **GPU → T4** via *Runtime → Change runtime type* before running.  
> **GCE VM:** See the **VM Quick-Start** section below, or just run `vm_setup.sh` via SSH.

---
## 0. VM Quick-Start (skip this on Colab)

If you're running on a **fresh GCE VM**, create it with the `gcloud` command below,
then SSH in and run the setup script. Skip to **Step 2 (Data)** after setup.

```bash
# ── Create a GCE Spot VM with T4 GPU (~$0.11/hr) ────────────────
gcloud compute instances create ids-training \
    --zone=us-central1-a \
    --machine-type=n1-standard-8 \
    --accelerator=type=nvidia-tesla-t4,count=1 \
    --maintenance-policy=TERMINATE \
    --provisioning-model=SPOT \
    --boot-disk-size=100GB \
    --image-family=ubuntu-2204-lts \
    --image-project=ubuntu-os-cloud \
    --metadata=startup-script='#!/bin/bash
echo "VM ready. SSH in and run vm_setup.sh"'

# ── SSH into the VM ──────────────────────────────────────────────
gcloud compute ssh ids-training --zone=us-central1-a

# ── Run the setup script (installs drivers, CUDA, Python, repo) ─
curl -sL https://raw.githubusercontent.com/samaraweeramethun-eng/IDS_Interpretability/main/vm_setup.sh | bash

# ── Upload your dataset ─────────────────────────────────────────
# From your LOCAL machine:
gcloud compute scp cicids2017.csv ids-training:~/IDS_Interpretability/data/cicids2017/ --zone=us-central1-a

# ── Run the full pipeline (headless, ~65 min on T4) ─────────────
cd ~/IDS_Interpretability && source .venv/bin/activate
python scripts/run_full_pipeline.py --data data/cicids2017/cicids2017.csv

# ── Download results ─────────────────────────────────────────────
# From your LOCAL machine:
gcloud compute scp --recurse ids-training:~/IDS_Interpretability/artifacts/ ./artifacts/ --zone=us-central1-a

# ── Delete VM when done ──────────────────────────────────────────
gcloud compute instances delete ids-training --zone=us-central1-a
```

---
## 1. Setup — Clone & Install

In [ ]:
import sys, os, subprocess

# ── Detect environment ───────────────────────────────────────────────
IN_COLAB = 'google.colab' in sys.modules or os.path.exists('/content')
IN_VM = not IN_COLAB
print(f"Environment: {'Google Colab' if IN_COLAB else 'VM / Local'}")

# ── Install NVIDIA driver + CUDA if needed (VM only) ────────────────
if IN_VM and not os.path.exists('/usr/local/cuda'):
    print("Installing NVIDIA driver + CUDA 12.1 (this takes a few minutes)...")
    cmds = [
        "sudo apt-get update -qq",
        "sudo apt-get install -y -qq build-essential curl wget",
        "wget -q https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.1-1_all.deb -O /tmp/cuda-keyring.deb",
        "sudo dpkg -i /tmp/cuda-keyring.deb",
        "sudo apt-get update -qq",
        "sudo apt-get install -y -qq cuda-toolkit-12-1 cuda-drivers",
    ]
    for cmd in cmds:
        subprocess.run(cmd, shell=True, check=True)
    os.environ['PATH'] = '/usr/local/cuda-12.1/bin:' + os.environ.get('PATH', '')
    os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-12.1/lib64:' + os.environ.get('LD_LIBRARY_PATH', '')
    print("✓ CUDA installed")

# ── Clone repository ─────────────────────────────────────────────────
REPO_DIR = '/content/IDS_Interpretability' if IN_COLAB else os.path.expanduser('~/IDS_Interpretability')
if not os.path.exists(REPO_DIR):
    !git clone https://github.com/samaraweeramethun-eng/IDS_Interpretability.git {REPO_DIR}
else:
    print(f"Repo exists at {REPO_DIR} — pulling latest...")
    !cd {REPO_DIR} && git pull origin main

os.chdir(REPO_DIR)
print(f"Working directory: {os.getcwd()}")

# ── Install PyTorch + dependencies ───────────────────────────────────
!pip install -q torch --index-url https://download.pytorch.org/whl/cu121
!pip install -q -r requirements.txt
!pip install -q -e .
!pip install -q psutil

# ── Fallback: add src/ to sys.path ───────────────────────────────────
SRC_DIR = os.path.join(os.getcwd(), 'src')
if SRC_DIR not in sys.path:
    sys.path.insert(0, SRC_DIR)

# ── Verify ───────────────────────────────────────────────────────────
import methun_research
print(f"\n✓ methun_research imported from: {methun_research.__file__}")

import torch
print(f"✓ PyTorch {torch.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    free, total = torch.cuda.mem_get_info(0)
    print(f"✓ VRAM: {total/1024**3:.1f} GB total, {free/1024**3:.1f} GB free")
else:
    print("⚠ WARNING: No GPU detected — training will be very slow!")

---
## 2. Data — Load CICIDS2017

Choose **one** of the options below:

- **Option A:** Mount Google Drive (Colab)  
- **Option B:** Upload a CSV interactively (Colab)  
- **Option C:** Copy from GCS bucket (VM — `gsutil cp`)  
- **Option D:** SCP from local machine (VM — `gcloud compute scp`)  
- **Option E (default):** Use the included 5 000-row sample for a quick smoke test

In [ ]:
import os

# ── Option A: Mount Google Drive (Colab only) ───────────────────────
# Uncomment if your dataset is on Google Drive.

# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
# DRIVE_CSV = '/content/drive/MyDrive/cicids2017/cicids2017.csv'
# !mkdir -p data/cicids2017
# !ln -sf "$DRIVE_CSV" data/cicids2017/cicids2017.csv

# ── Option B: Upload from local machine (Colab only) ────────────────
# Uncomment to upload interactively.

# from google.colab import files
# uploaded = files.upload()  # select your cicids2017.csv
# !mkdir -p data/cicids2017
# !mv "{list(uploaded.keys())[0]}" data/cicids2017/cicids2017.csv

# ── Option C: GCS bucket (VM recommended) ───────────────────────────
# Uncomment and fill in your bucket name.

# !gsutil cp gs://YOUR_BUCKET/cicids2017.csv data/cicids2017/cicids2017.csv

# ── Option D: SCP (VM) ──────────────────────────────────────────────
# From your LOCAL machine run:
#   gcloud compute scp cicids2017.csv ids-training:~/IDS_Interpretability/data/cicids2017/ --zone=us-central1-a

# ── Auto-detect dataset ─────────────────────────────────────────────
DATA_PATH = 'data/cicids2017/cicids2017_sample.csv'
if os.path.exists('data/cicids2017/cicids2017.csv'):
    DATA_PATH = 'data/cicids2017/cicids2017.csv'

print(f'Using dataset: {DATA_PATH}')
print(f'Size: {os.path.getsize(DATA_PATH) / 1024**2:.1f} MB')

import pandas as pd
df_peek = pd.read_csv(DATA_PATH, nrows=5)
print(f'Columns: {len(df_peek.columns)}')
label_col = [c for c in df_peek.columns if 'label' in c.lower()][0]
counts = pd.read_csv(DATA_PATH, usecols=[label_col])[label_col].value_counts()
print(f'\nLabel distribution:\n{counts}')

---
## 3. Configure Training

Adjust hyperparameters below. The defaults are tuned for a **T4 GPU** with the full dataset.  
If you are using the 5 000-row sample, the `_sample` overrides will automatically apply.

In [ ]:
import os, sys, warnings
os.environ['TORCHDYNAMO_DISABLE'] = '1'
warnings.filterwarnings('ignore')

# Ensure methun_research is importable (survives kernel restart)
SRC_DIR = os.path.join(os.getcwd(), 'src')
if SRC_DIR not in sys.path:
    sys.path.insert(0, SRC_DIR)

from methun_research.config import CNNTransformerConfig, EnhancedConfig

USE_SAMPLE = 'sample' in DATA_PATH

# ── CNN-Transformer Config (optimised for T4 GPU) ────────────────────
cnn_cfg = CNNTransformerConfig(
    input_path=DATA_PATH,
    output_dir='artifacts',
    epochs=25 if not USE_SAMPLE else 5,
    batch_size=1024 if not USE_SAMPLE else 64,
    val_batch_size=2048 if not USE_SAMPLE else 128,
    lr=2e-3 if not USE_SAMPLE else 1.5e-3,
    num_workers=2,
    d_model=192 if not USE_SAMPLE else 64,
    conv_channels=96 if not USE_SAMPLE else 32,
    num_layers=3 if not USE_SAMPLE else 1,
    num_heads=8 if not USE_SAMPLE else 4,
    d_ff=768 if not USE_SAMPLE else 256,
    dropout=0.2,
    val_size=0.1,
    test_size=0.2,
    ig_steps=32 if not USE_SAMPLE else 8,
    ig_samples=512 if not USE_SAMPLE else 128,
    max_train_samples=500_000 if not USE_SAMPLE else 0,
)

# ── Enhanced Transformer Config (optimised for T4 GPU) ───────────────
enh_cfg = EnhancedConfig(
    input_path=DATA_PATH,
    output_dir='artifacts',
    epochs=35 if not USE_SAMPLE else 5,
    batch_size=1024 if not USE_SAMPLE else 64,
    val_batch_size=2048 if not USE_SAMPLE else 128,
    lr=2e-3,
    num_workers=2,
    d_model=160 if not USE_SAMPLE else 64,
    num_layers=4 if not USE_SAMPLE else 2,
    heads=10 if not USE_SAMPLE else 4,
    d_ff=640 if not USE_SAMPLE else 256,
    dropout=0.15,
    group_size=8,
    use_swa=True if not USE_SAMPLE else False,
    use_mixup=True if not USE_SAMPLE else False,
    max_train_samples=500_000 if not USE_SAMPLE else 0,
)

print(f'Mode: {"SAMPLE (small model)" if USE_SAMPLE else "FULL DATASET (production model)"}')
print(f'Data split: {100*(1-cnn_cfg.val_size-cnn_cfg.test_size):.0f}% train / {100*cnn_cfg.val_size:.0f}% val / {100*cnn_cfg.test_size:.0f}% test')
print(f'\nCNN-Transformer: {cnn_cfg.epochs} epochs, d_model={cnn_cfg.d_model}, batch={cnn_cfg.batch_size}')
print(f'Enhanced:        {enh_cfg.epochs} epochs, d_model={enh_cfg.d_model}, batch={enh_cfg.batch_size}')
print(f'Max train samples: {cnn_cfg.max_train_samples or "unlimited"}')

---
## 4. Train CNN-Transformer

This trains the CNN + Transformer hybrid and automatically generates:
- **Integrated Gradients** feature attributions
- **Grad-CAM** activation maps over feature positions

In [ ]:
import gc, time, torch
from methun_research.training.cnn_trainer import train_cnn_transformer

# Show RAM before training
import psutil
ram = psutil.virtual_memory()
print(f'System RAM: {ram.used/1024**3:.1f} / {ram.total/1024**3:.1f} GB ({ram.percent}% used)')
if torch.cuda.is_available():
    free, total = torch.cuda.mem_get_info(0)
    print(f'GPU VRAM:   {(total-free)/1024**3:.1f} / {total/1024**3:.1f} GB used')

gc.collect()
print('\nTraining CNN-Transformer...')
t0 = time.time()
cnn_path = train_cnn_transformer(cnn_cfg)

elapsed = (time.time() - t0) / 60
print(f'\nDone in {elapsed:.1f} min  ->  {cnn_path}')

# Show resource usage after training
ram = psutil.virtual_memory()
print(f'\nSystem RAM: {ram.used/1024**3:.1f} / {ram.total/1024**3:.1f} GB ({ram.percent}% used)')
if torch.cuda.is_available():
    free, total = torch.cuda.mem_get_info(0)
    print(f'GPU VRAM:   {(total-free)/1024**3:.1f} / {total/1024**3:.1f} GB used')
    print(f'GPU peak:   {torch.cuda.max_memory_allocated(0)/1024**3:.2f} GB')

---
## 5. Train Enhanced Transformer

In [ ]:
import gc, torch
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

from methun_research.training.enhanced_trainer import train_enhanced

print('Training Enhanced Transformer...')
t0 = time.time()
enh_path = train_enhanced(enh_cfg)

elapsed = (time.time() - t0) / 60
print(f'\nDone in {elapsed:.1f} min  ->  {enh_path}')

if torch.cuda.is_available():
    free, total = torch.cuda.mem_get_info(0)
    print(f'GPU VRAM:   {(total-free)/1024**3:.1f} / {total/1024**3:.1f} GB used')
    print(f'GPU peak:   {torch.cuda.max_memory_allocated(0)/1024**3:.2f} GB')

---
## 6. SHAP Analysis — CNN-Transformer

Runs `GradientExplainer` on the CNN-Transformer checkpoint.  
Produces: global importance CSV, summary beeswarm plot, single-sample waterfall plot.

In [ ]:
from methun_research.interpretability.shap_runner import run_shap

os.makedirs('artifacts/shap', exist_ok=True)

SHAP_BG   = 2000 if not USE_SAMPLE else 200
SHAP_EVAL = 2000 if not USE_SAMPLE else 200
SHAP_POOL = 150_000 if not USE_SAMPLE else 500

print('Running SHAP on CNN-Transformer...')
t0 = time.time()
shap_csv = run_shap(
    checkpoint_path=cnn_path,
    data_path=DATA_PATH,
    output_dir='artifacts/shap',
    background_size=SHAP_BG,
    eval_size=SHAP_EVAL,
    eval_pool=SHAP_POOL,
    chunk_size=256,
)
print(f'\nDone in {(time.time()-t0)/60:.1f} min  ->  {shap_csv}')

---
## 7. SHAP Analysis — Enhanced Transformer

In [ ]:
if enh_path:
    print('Running SHAP on Enhanced Transformer...')
    t0 = time.time()
    shap_enh_csv = run_shap(
        checkpoint_path=enh_path,
        data_path=DATA_PATH,
        output_dir='artifacts/shap_enhanced',
        background_size=SHAP_BG,
        eval_size=SHAP_EVAL,
        eval_pool=SHAP_POOL,
        chunk_size=256,
    )
    print(f'\nDone in {(time.time()-t0)/60:.1f} min  ->  {shap_enh_csv}')
else:
    print('No Enhanced Transformer checkpoint — skipping.')

---
## 8. Visualise XAI Results

Compare feature rankings across all four XAI methods.

In [ ]:
import pandas as pd
from IPython.display import display, HTML

TOP_K = 15

def load_ranking(path, val_col, method_name):
    if not os.path.exists(path):
        print(f'  [{method_name}] Not found: {path}')
        return None
    df = pd.read_csv(path)
    df = df.sort_values(val_col, ascending=False).head(TOP_K).reset_index(drop=True)
    df.index += 1
    df.columns = ['Feature', method_name]
    return df

rankings = [
    load_ranking('artifacts/cnn_transformer_integrated_gradients.csv',
                 'avg_abs_integrated_grad', 'Integrated Gradients'),
    load_ranking('artifacts/cnn_transformer_grad_cam.csv',
                 'grad_cam_importance', 'Grad-CAM'),
    load_ranking('artifacts/shap/shap_global_importance_attack.csv',
                 'mean_abs_shap', 'SHAP (CNN-Trans)'),
]

# Enhanced SHAP if available
r = load_ranking('artifacts/shap_enhanced/shap_global_importance_attack.csv',
                 'mean_abs_shap', 'SHAP (Enhanced)')
if r is not None:
    rankings.append(r)

rankings = [r for r in rankings if r is not None]
if rankings:
    combined = pd.concat([r.set_index(r.index) for r in rankings], axis=1)
    display(HTML(f'<h3>Top {TOP_K} Features — All XAI Methods</h3>'))
    display(combined)
else:
    print('No ranking files found.')

In [ ]:
# Display the generated plots
from IPython.display import Image, display

plots = [
    ('Grad-CAM Feature Importance', 'artifacts/grad_cam_importance.png'),
    ('SHAP Summary (CNN-Transformer)', 'artifacts/shap/shap_summary_attack.png'),
    ('SHAP Waterfall (CNN-Transformer)', 'artifacts/shap/shap_waterfall_attack.png'),
    ('SHAP Summary (Enhanced)', 'artifacts/shap_enhanced/shap_summary_attack.png'),
    ('SHAP Waterfall (Enhanced)', 'artifacts/shap_enhanced/shap_waterfall_attack.png'),
]

for title, path in plots:
    if os.path.exists(path):
        print(f'\n--- {title} ---')
        display(Image(filename=path, width=700))
    else:
        print(f'[skip] {title} — file not found')

---
## 9. Save Artifacts to Google Drive (Optional)

Copy all checkpoints and XAI outputs to your Drive for later use.

In [ ]:
# Uncomment to save artifacts to Google Drive
# import shutil
# from google.colab import drive
# drive.mount('/content/drive', force_remount=False)

# DEST = '/content/drive/MyDrive/methun_research_artifacts'
# shutil.copytree('artifacts', DEST, dirs_exist_ok=True)
# print(f'Artifacts saved to {DEST}')

---
## 10. List All Generated Artifacts

In [ ]:
print('All generated artifacts:')
for root, dirs, files in os.walk('artifacts'):
    for f in sorted(files):
        fp = os.path.join(root, f)
        sz = os.path.getsize(fp)
        if sz > 1024*1024:
            print(f'  {fp}  ({sz/1024**2:.1f} MB)')
        else:
            print(f'  {fp}  ({sz/1024:.1f} KB)')